In [1]:
# Import des bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix, classification_report
from tensorflow.keras.callbacks import EarlyStopping
import cv2
import os

# Vérification de la disponibilité des GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Définition des hyperparamètres globaux
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32
EPOCHS = 10
FOLDS = 3
DATA_DIR = '../../../chest_xray/train'

# 1. Objectif et hypothèses
# (écrivez vos objectifs et hypothèses dans une cellule de texte Markdown si vous utilisez Jupyter Notebook)

# 2. Préparation des données
def load_data(data_dir):
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1
    )
    
    train_data = datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='grayscale',
        subset='training'
    )
    
    val_data = datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='grayscale',
        subset='validation'
    )
    
    return train_data, val_data

train_data, val_data = load_data(DATA_DIR)

# 3. Définition du modèle CNN
def create_model(num_conv_layers):
    model = models.Sequential()
    model.add(layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1)))  # Conversion en niveaux de gris
    
    for i in range(num_conv_layers):
        model.add(layers.Conv2D(16 * (2**i), (3, 3), activation='relu')) 
        model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
    return model

# 4. Entraînement du modèle avec validation croisée
def k_fold_cross_validation(num_conv_layers, train_data, val_data):
    kf = KFold(n_splits=FOLDS, shuffle=True)
    accuracy_list, auc_list, all_histories = [], [], []

    for train_index, val_index in kf.split(train_data):
        model = create_model(num_conv_layers)
        early_stopping = EarlyStopping(monitor='val_loss', patience=3)
        
        history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS, callbacks=[early_stopping], verbose=0)
        all_histories.append(history.history)
        
        val_loss, val_accuracy, val_auc = model.evaluate(val_data)
        accuracy_list.append(val_accuracy)
        auc_list.append(val_auc)

    return np.mean(accuracy_list), np.mean(auc_list), all_histories

# Test avec différents nombres de couches de convolution
conv_layers_list = [2, 3]
results = {}

for num_layers in conv_layers_list:
    accuracy, roc_auc, histories = k_fold_cross_validation(num_layers, train_data, val_data)
    results[num_layers] = {'Accuracy': accuracy, 'AUC': roc_auc}
    print(f'Layers: {num_layers}, Accuracy: {accuracy:.4f}, AUC: {roc_auc:.4f}')

# 5. Évaluation finale sur les données de test
def evaluate_on_test_data(model, test_data):
    val_loss, val_accuracy, val_auc = model.evaluate(test_data)
    print(f"\nTest Loss: {val_loss:.4f}")
    print(f"Test Accuracy: {val_accuracy:.4f}")
    print(f"Test AUC: {val_auc:.4f}")

# 6. Interprétation des prédictions avec Grad-CAM
def grad_cam(model, img_array, last_conv_layer_name="conv2d"):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    conv_outputs *= pooled_grads

    heatmap = tf.reduce_mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    heatmap = heatmap.numpy()
    return heatmap

# Fonction pour superposer la carte thermique sur l'image
def superimpose_heatmap(img_path, heatmap):
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)
    return superimposed_img

# 7. Conclusion
def display_results(results):
    results_df = pd.DataFrame(results).T
    print("\n--- Résultats Finaux ---")
    print(results_df)
    for num_layers, metrics in results.items():
        print(f"\n--- Détails pour {num_layers} Couches de Convolution ---")
        print(f"  Précision Moyenne: {metrics['Accuracy']:.4f}")
        print(f"  AUC Moyenne: {metrics['AUC']:.4f}")

def plot_results(histories, conv_layers_list):
    for num_layers, history in zip(conv_layers_list, histories):
        plt.plot(history['accuracy'], label='Accuracy')
        plt.plot(history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Model Accuracy for {num_layers} Convolutional Layers')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(loc='upper left')
        plt.grid()
        plt.show()

        plt.plot(history['loss'], label='Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.title(f'Model Loss for {num_layers} Convolutional Layers')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc='upper left')
        plt.grid()
        plt.show()

display_results(results)
plot_results([history for _, history in results.items()], conv_layers_list)


Num GPUs Available:  0
Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.


C:\Users\vikne\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 658ms/step - AUC: 0.9686 - accuracy: 0.9249 - loss: 0.2027
33/33 ━━━━━━━━━━━━━━━━━━━━ 13s 389ms/step - AUC: 0.9757 - accuracy: 0.9258 - loss: 0.1958
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 353ms/step - AUC: 0.9660 - accuracy: 0.9040 - loss: 0.2191
Layers: 2, Accuracy: 0.9259, AUC: 0.9751
33/33 ━━━━━━━━━━━━━━━━━━━━ 14s 420ms/step - AUC: 0.9869 - accuracy: 0.9426 - loss: 0.1367
33/33 ━━━━━━━━━━━━━━━━━━━━ 14s 421ms/step - AUC: 0.9678 - accuracy: 0.9047 - loss: 0.2220
33/33 ━━━━━━━━━━━━━━━━━━━━ 15s 437ms/step - AUC: 0.9849 - accuracy: 0.9258 - loss: 0.1666
Layers: 3, Accuracy: 0.9332, AUC: 0.9825

--- Résultats Finaux ---
   Accuracy       AUC
2  0.925855  0.975126
3  0.933206  0.982493

--- Détails pour 2 Couches de Convolution ---
  Précision Moyenne: 0.9259
  AUC Moyenne: 0.9751

--- Détails pour 3 Couches de Convolution ---
  Précision Moyenne: 0.9332
  AUC Moyenne: 0.9825


KeyError: 'accuracy'